# Not Scaled Data

In [33]:
import pandas as pd
xls = pd.ExcelFile("Фрустрация.xlsx")
data = pd.read_excel(xls, xls.sheet_names)    

In [34]:
keys = [' E', ' E1', ' e_', ' M', ' M1', ' m_', ' I', ' I1', ' i_']

keys1 = [[' E', ' E1', ' e_'],
         [' M', ' M1', ' m_'],
         [' I', ' I1', ' i_']]

keys2 = [[' E', ' M', ' I'],
        [' E1', ' M1', ' I1'],
        [' e_', ' m_', ' i_']]
def get_y(keys, data, init_keys = keys):
    y = []
    for ind, i in enumerate(init_keys):
        for ind1, j in enumerate(keys):
            if i in j:
                y.append(ind1 * np.ones(len(data[i])))
    return np.hstack(y)

In [35]:
data[keys[0]].columns

Index(['Unnamed: 0', 'Число восклицательных знаков',
       'Число вопросительных знаков', 'Число слов', 'Число символов',
       'Число уникальных слов', 'Число клауз', 'Число предложений',
       'Число знаков препинания',
       'Средняя длина предложения (в количестве слов)',
       ...
       'Синтакс. связь: nummod', 'Синтакс. связь: nummod:gov',
       'Синтакс. связь: obj', 'Синтакс. связь: obl', 'Синтакс. связь: orphan',
       'Синтакс. связь: parataxis', 'Синтакс. связь: punct',
       'Синтакс. связь: root', 'Синтакс. связь: xcomp',
       'Синтакс. связь: compound'],
      dtype='object', length=243)

In [268]:
import numpy as np
from sklearn.preprocessing import scale
X = np.vstack([data[i].values for i in keys])
X = X[:, 1:]
y1 = get_y(keys1, data)
y2 = get_y(keys2, data)

In [269]:
print("Число слов", X[:, 2].mean())

Число слов 36.49063670411985


In [270]:
print("Число предложений", X[:, 6].mean())

Число предложений 3.4569288389513106


In [271]:
print("Число символов", X[:, 3].mean())

Число символов 188.49812734082397


In [272]:
X = scale(X)

## The First Union

### 1. Not binarized data

In [292]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.utils import shuffle

X_, y = shuffle(X, y1)
print(X_.shape)
model = LogisticRegression(penalty = "l1", solver = "liblinear", class_weight = "balanced", multi_class = "ovr")
cross_val_score(model, X_, y, cv = 3, scoring = "f1_macro").mean()

(267, 242)


0.43394302456802464

In [293]:
from sklearn.linear_model import LogisticRegression

X_ = PCA(1 - 1e-3).fit(X_).transform(X_)
print(X_.shape)
model = LogisticRegression(penalty = "l1", solver = "liblinear", class_weight = "balanced", multi_class = "ovr")
cross_val_score(model, X_, y, cv = 3, scoring = "f1_macro").mean()

(267, 156)


0.39070944088433307

In [295]:
from sklearn.ensemble import GradientBoostingClassifier

model = GradientBoostingClassifier(n_estimators = 1000, max_depth = 10)

cross_val_score(model, X_, y, cv = 3, scoring = "f1_macro").mean()

0.3182916672146167

### 2. Binarized data

In [296]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.utils import shuffle

X_, y = shuffle(X_bin, y1)
print(X_.shape)
model = LogisticRegression(penalty = "l1", solver = "liblinear", class_weight = "balanced", multi_class = "ovr")
cross_val_score(model, X_, y, cv = 3, scoring = "f1_macro").mean()

(267, 484)


0.3693661227392415

In [297]:
from sklearn.linear_model import LogisticRegression

X_ = PCA(1 - 1e-3).fit(X_).transform(X_)
print(X_.shape)
model = LogisticRegression(penalty = "l1", solver = "liblinear", class_weight = "balanced", multi_class = "ovr")
cross_val_score(model, X_, y, cv = 3, scoring = "f1_macro").mean()

(267, 203)


0.3023163555211201

In [298]:
from sklearn.ensemble import GradientBoostingClassifier

model = GradientBoostingClassifier(n_estimators = 1000, max_depth = 10)

cross_val_score(model, X_, y, cv = 3, scoring = "f1_macro").mean()

/home/elias/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/elias/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


0.3116150044761569

## The Second Union

### 1. Not binarized data

In [320]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.utils import shuffle

X_, y = shuffle(X, y2)
print(X_.shape)
model = LogisticRegression(penalty = "l1", solver = "liblinear", class_weight = "balanced", multi_class = "ovr")
cross_val_score(model, X_, y, cv = 3, scoring = "f1_macro").mean()

(267, 242)


0.3493170155919523

In [321]:
from sklearn.linear_model import LogisticRegression

X_ = PCA(1 - 1e-3).fit(X_).transform(X_)
print(X_.shape)
model = LogisticRegression(penalty = "l1", solver = "liblinear", class_weight = "balanced", multi_class = "ovr")
cross_val_score(model, X_, y, cv = 3, scoring = "f1_macro").mean()

(267, 156)


0.366996217602315

In [322]:
from sklearn.ensemble import GradientBoostingClassifier

model = GradientBoostingClassifier(n_estimators = 10, max_depth = 5)

cross_val_score(model, X_, y, cv = 3, scoring = "f1_macro").mean()

0.34228207437427294

### 2. Binarized data

In [326]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.utils import shuffle

X_, y = shuffle(X_bin, y2)
print(X_.shape)
model = LogisticRegression(penalty = "l1", solver = "liblinear", class_weight = "balanced", multi_class = "ovr")
cross_val_score(model, X_, y, cv = 3, scoring = "f1_macro").mean()

(267, 484)


0.35510332478836415

In [327]:
from sklearn.linear_model import LogisticRegression

X_ = PCA(1 - 1e-3).fit(X_).transform(X_)
print(X_.shape)
model = LogisticRegression(penalty = "l1", solver = "liblinear", class_weight = "balanced", multi_class = "ovr")
cross_val_score(model, X_, y, cv = 3, scoring = "f1_macro").mean()

(267, 203)


0.3216505447178342

In [328]:
from sklearn.ensemble import GradientBoostingClassifier

model = GradientBoostingClassifier(n_estimators = 1000, max_depth = 10)

cross_val_score(model, X_, y, cv = 3, scoring = "f1_macro").mean()

/home/elias/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


0.30451303293029913

# Scaled Data

In [329]:
import pandas as pd
xls = pd.ExcelFile("Фрустрация.xlsx")
data = pd.read_excel(xls, xls.sheet_names)    

In [330]:
keys = [' E', ' E1', ' e_', ' M', ' M1', ' m_', ' I', ' I1', ' i_']

keys1 = [[' E', ' E1', ' e_'],
         [' M', ' M1', ' m_'],
         [' I', ' I1', ' i_']]

keys2 = [[' E', ' M', ' I'],
        [' E1', ' M1', ' I1'],
        [' e_', ' m_', ' i_']]
def get_y(keys, data, init_keys = keys):
    y = []
    for ind, i in enumerate(init_keys):
        for ind1, j in enumerate(keys):
            if i in j:
                y.append(ind1 * np.ones(len(data[i])))
    return np.hstack(y)

In [331]:
data[keys[0]].columns

Index(['Unnamed: 0', 'Число восклицательных знаков',
       'Число вопросительных знаков', 'Число слов', 'Число символов',
       'Число уникальных слов', 'Число клауз', 'Число предложений',
       'Число знаков препинания',
       'Средняя длина предложения (в количестве слов)',
       ...
       'Синтакс. связь: nummod', 'Синтакс. связь: nummod:gov',
       'Синтакс. связь: obj', 'Синтакс. связь: obl', 'Синтакс. связь: orphan',
       'Синтакс. связь: parataxis', 'Синтакс. связь: punct',
       'Синтакс. связь: root', 'Синтакс. связь: xcomp',
       'Синтакс. связь: compound'],
      dtype='object', length=243)

In [332]:
import numpy as np
from sklearn.preprocessing import scale
X = np.vstack([data[i].values for i in keys])
X = X[:, 1:]
X1 = X/X[:, 6][:, None]
X = np.hstack([X, X1])
y1 = get_y(keys1, data)
y2 = get_y(keys2, data)

In [333]:
print("Число слов", X[:, 2].mean())

Число слов 36.49063670411985


In [334]:
print("Число предложений", X[:, 6].mean())

Число предложений 3.4569288389513106


In [335]:
print("Число символов", X[:, 3].mean())

Число символов 188.49812734082397


In [336]:
X = scale(X)

## The First Union

### 1. Not binarized data

In [339]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.utils import shuffle

X_, y = shuffle(X, y1)
print(X_.shape)
model = LogisticRegression(penalty = "l1", solver = "liblinear", class_weight = "balanced", multi_class = "ovr")
cross_val_score(model, X_, y, cv = 3, scoring = "f1_macro").mean()

(267, 484)


0.36229172215994154

In [340]:
from sklearn.linear_model import LogisticRegression

X_ = PCA(1 - 1e-3).fit(X_).transform(X_)
print(X_.shape)
model = LogisticRegression(penalty = "l1", solver = "liblinear", class_weight = "balanced", multi_class = "ovr")
cross_val_score(model, X_, y, cv = 3, scoring = "f1_macro").mean()

(267, 210)


0.3847580460729462

In [341]:
from sklearn.ensemble import GradientBoostingClassifier

model = GradientBoostingClassifier(n_estimators = 1000, max_depth = 10)

cross_val_score(model, X_, y, cv = 3, scoring = "f1_macro").mean()

/home/elias/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


0.3401598969408239

### 2. Binarized data

In [342]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.utils import shuffle

X_, y = shuffle(X_bin, y1)
print(X_.shape)
model = LogisticRegression(penalty = "l1", solver = "liblinear", class_weight = "balanced", multi_class = "ovr")
cross_val_score(model, X_, y, cv = 3, scoring = "f1_macro").mean()

(267, 484)


0.3099135622187438

In [343]:
from sklearn.linear_model import LogisticRegression

X_, y = shuffle(X_bin, y1)
X_ = PCA(1 - 1e-3).fit(X_).transform(X_)
print(X_.shape)
model = LogisticRegression(penalty = "l1", solver = "liblinear", class_weight = "balanced", multi_class = "ovr")
cross_val_score(model, X_, y, cv = 3, scoring = "f1_macro").mean()

(267, 203)


0.2944286069518886

In [344]:
from sklearn.ensemble import GradientBoostingClassifier

model = GradientBoostingClassifier(n_estimators = 1000, max_depth = 10)

cross_val_score(model, X_, y, cv = 3, scoring = "f1_macro").mean()

0.3564686308931833

## The Second Union

### 1. Not binarized data

In [347]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.utils import shuffle

X_, y = shuffle(X, y2)
print(X_.shape)
model = LogisticRegression(penalty = "l1", solver = "liblinear", class_weight = "balanced", multi_class = "ovr")
cross_val_score(model, X_, y, cv = 3, scoring = "f1_macro").mean()

(267, 484)


0.35439179134432214

In [348]:
from sklearn.linear_model import LogisticRegression

X_, y = shuffle(X, y2)
X_ = PCA(1 - 1e-3).fit(X_).transform(X_)
print(X_.shape)
model = LogisticRegression(penalty = "l1", solver = "liblinear", class_weight = "balanced", multi_class = "ovr")
cross_val_score(model, X_, y, cv = 3, scoring = "f1_macro").mean()

(267, 210)


0.38024353390771953

In [349]:
from sklearn.ensemble import GradientBoostingClassifier

model = GradientBoostingClassifier(n_estimators = 1000, max_depth = 10)

cross_val_score(model, X_, y, cv = 3, scoring = "f1_macro").mean()

0.29231661274542103

### 2. Binarized data

In [350]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.utils import shuffle

X_, y = shuffle(X_bin, y2)
print(X_.shape)
model = LogisticRegression(penalty = "l1", solver = "liblinear", class_weight = "balanced", multi_class = "ovr")
cross_val_score(model, X_, y, cv = 3, scoring = "f1_macro").mean()

(267, 484)


0.3032155332411995

In [351]:
from sklearn.linear_model import LogisticRegression

X_, y = shuffle(X_bin, y2)
X_ = PCA(1 - 1e-3).fit(X_).transform(X_)
print(X_.shape)
model = LogisticRegression(penalty = "l1", solver = "liblinear", class_weight = "balanced", multi_class = "ovr")
cross_val_score(model, X_, y, cv = 3, scoring = "f1_macro").mean()

(267, 203)


0.3375467676948692

In [352]:
from sklearn.ensemble import GradientBoostingClassifier

model = GradientBoostingClassifier(n_estimators = 1000, max_depth = 10)

cross_val_score(model, X_, y, cv = 3, scoring = "f1_macro").mean()

0.3486800170253408